# S3DIS inference
This notebook lets you run inference on the S3DIS Test Fold, from a pretrained model's checkpoint on your machine. The output will be a confusion matrix which you can save locally on your machine. For this, you must have done the following:
- downloaded S3DIS dataset (if you haven't, instantiating the dataset will launch it for you though)
- trained a model (which produced a checkpoint directory) or downloaded our pretrained weights

Note this notebook should work for multimodal (2D+3D) and unimodal (3D only) experiments.

In [ ]:
# Select you GPU
I_GPU = 0

In [ ]:
# Uncomment to use autoreload
# %load_ext autoreload
# %autoreload 2

import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read
from torch_points3d.trainer import Trainer
import warnings
warnings.filterwarnings('ignore')

DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

_ = torch.cuda.is_available()
_ = torch.cuda.memory_allocated()
torch.cuda.set_device(I_GPU)

In [ ]:
# Set your parameters
checkpoint_dir = '/directory/containing/your/checkpoint/file'
result_dir = '/directory/where/to/save/inference/metrics'
model_name = 'Res16UNet34-L4-early'                                   # adapt if you use another model in your checkpoint
exp_name = None                                                       # you may give a name to the experiment
exp_name = model_name if exp_name is None else exp_name
n_votes = 1                                                           # number of inferences per spherical sample. For multi-inference voting with inference-time augmentation
batch_size = 8                                                        # increase if your device allows it
full_res = True                                                       # predictions will be made on the raw point cloud, at full resolution
n_tries = 1                                                           # number of inferences 
num_workers = 4                                                       # increase if your machine allows it

In [ ]:
print(f'Inference on S3DIS exp={exp_name}')

# These are the arguments passed to Hydra. You could run the same thing 
# from CLI using `eval.py` with the command `python eval.py [...]`
overrides = [
    f'model_name={model_name}',
    f"checkpoint_dir={checkpoint_dir}",
    f'voting_runs={n_votes}',
    f'tracker_options.full_res={full_res}',
    'precompute_multi_scale=False',
    f'num_workers={num_workers}',
    f'batch_size={batch_size}',
    f'cuda={I_GPU}',
    'weight_name=last',
]

# Parse the arguments with Hydra and OmegaConf
cfg = hydra_read(overrides, config_name='eval')
OmegaConf.set_struct(cfg, False)

# Create the Trainer instance from your checkpoint
trainer = Trainer(cfg)

# Update the val and test transforms to match train transforms for 
# inference-time augmentation
trainer._dataset.test_dataset[0].transform = trainer._dataset.train_dataset.transform
if trainer._model.is_multimodal:
    trainer._dataset.test_dataset[0].transform_image = trainer._dataset.train_dataset.transform_image
    trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
    trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 * 2)
    trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
    trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.8, 1.2]
    trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.8, 1.2]
    trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.8, 1.2]

# Run inference
for i_try in range(n_tries):
    print(f'  run {i_try}/{n_tries}...')
    trainer.eval(stage_name='test')
    cm = trainer._tracker._full_confusion_matrix
    torch.save(cm, f'{result_dir}/{exp_name}_votes-{trainer._cfg.voting_runs}_{i_try}.pt')
    print(f'  mIoU={cm.get_average_intersection_union() * 100:0.2f}')
    print()

Confusion matrices will be saved in your `result_dir`.